In [1]:
!pip install pyspark
!pip install python-dotenv
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 5.6 MB/s eta 0:00:00


In [2]:
import snowflake.connector
from google.colab import userdata

# Replace the placeholders with your actual Snowflake credentials
conn = snowflake.connector.connect(
    user = userdata.get('snowflake_userid'),
    password = userdata.get('snowflake_password'),
    account= userdata.get('snowflake_account'),
    warehouse='COMPUTE_WH',
    database='BIGDATA_GITHUB',
    schema='ANALYTICS',
    role='ACCOUNTADMIN'
)

In [6]:
cur = conn.cursor()
url = 'azure://matthewleffler1.blob.core.windows.net/kaggle-datasets/analytics/Top_license_trend/'

try:
    cur.execute("BEGIN;")
    cur.execute(f"""
        CREATE OR REPLACE STAGE BIGDATA_GITHUB.ANALYTICS.azure_parquet_stage_Top_License
          URL = '{url}'
          CREDENTIALS = (
            AZURE_SAS_TOKEN = '{userdata.get('AZURE_SAS_TOKEN')}'
          )
          FILE_FORMAT = (TYPE = PARQUET);
        """)
    cur.execute("COMMIT;")
    print(f"Successfullt created stage.")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Error creating database object: {e}")
finally:
    cur.close()

Successfullt created stage.


In [7]:
cur = conn.cursor()
table_name = "BIGDATA_GITHUB.ANALYTICS.Top_License"

try:
    cur.execute("BEGIN;")
    cur.execute(f"""
      CREATE OR REPLACE TABLE {table_name} (
      repo_license STRING,
      repo_language STRING,
      repo_count BIGINT,
      total_stars BIGINT
      );
          """)
    cur.execute("COMMIT;")
    print("Table created successfully.")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Error creating database object: {e}")
finally:
    cur.close()

Table created successfully.


In [8]:
cur = conn.cursor()

try:
    cur.execute("BEGIN;")
    cur.execute(f"""
      COPY INTO {table_name}
      FROM @azure_parquet_stage_Top_License
      FILE_FORMAT = (TYPE = PARQUET)
      MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
      ON_ERROR = CONTINUE;
          """)
    cur.execute("COMMIT;")
    print(f"Data loaded into {table_name} successfully.")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Error loading data: {e}")
finally:
    cur.close()

Data loaded into BIGDATA_GITHUB.ANALYTICS.Top_License successfully.
